# Unknown Number of Exponentials

If the structure of the model to fit is unknown, one can simply run the algorithm for a large $n$ and compute the singular values of $Y$. By looking at the singular values, we can infer how many of them are actually needed to fit most of the data. From this we can select a minimum $n$ and based in the singular values of $\hat{A}$ we can also deduce the structure of the model (how many exponentials, sines, cosines, if there are exponentials multiplied times $x$, etc.).

---

# Example

## Import [PyArma](https://pyarma.sourceforge.io/)

In [1]:
from pyarma import *
from pyarma_utils import cumtrapz, join_horizontal

## Generate data to fit

In [2]:
dx = 0.02
x = regspace(dx, dx, 1.5)
y = 5*exp(0.5*x) + 4*exp(-3*x) + 2*exp(-2*x)

## Compute `N` integrals of `y` and `N-1` powers of `x`

In [3]:
n = 10
iy = zeros(x.n_elem, n)
xp = zeros(x.n_elem, n-1)
iy[:,0] = cumtrapz(x, y)
xp[:,0] = ones(size(x))
for ii in range(1, n-1):
    iy[:, ii] = cumtrapz(x, iy[:, ii-1])
    xp[:, ii] = xp[:, ii-1] @ x
iy[:, n-1] = cumtrapz(x, iy[:, n-2])

## Compute Singular Values of `Y`

In [4]:
Y = join_horizontal(iy, xp)
ysv = svd(Y)[1]

## Scale Singular Values to Percentage

In [5]:
ysv_scaled = (100/sum(ysv)[0,0]) * ysv
ysv_scaled.print("ysv_scaled = ")

ysv_scaled = 
   6.9014e+01
   2.4043e+01
   4.5012e+00
   1.6753e+00
   6.4420e-01
   1.0649e-01
   1.4482e-02
   1.4624e-03
   1.0298e-04
   6.4525e-07
   4.2105e-09
   1.1134e-10
   1.6233e-12
   1.1949e-15
   3.5137e-16
   2.0894e-16
   5.7819e-17
   1.5788e-17
   3.1670e-18


## Select the N Principal Components

For example, select `N` above a threshold of $0.1%$.

In [6]:
thres = 0.1
n_ysv_scaled = mat(ysv_scaled[ find(ysv_scaled > thres) ])
N = n_ysv_scaled.n_elem / 2
print(N)

3.0


$N = 3$ ($6$ singular values)

In [7]:
covers = sum(n_ysv_scaled)
covers.print("covers = ")

covers = 
   99.9840


Covers `99.9%` of all components contributions